In [2]:
from pathlib import Path 
import math
import pandas as pd
import pickle
disease_dir = Path("/shared/share_mala/zj2398/mimic/regression_labels_nona/")
for parquet_file in disease_dir.glob("*.parquet"):
    disease_file = pd.read_parquet(parquet_file)
    # disease_file = disease_file.dropna(subset=["numeric_value"])
    # disease_file.to_parquet(parquet_file)
    print(len(disease_file))
    # # print(disease_file)
    mask_nan = disease_file["numeric_value"].notna()
    print(sum(mask_nan))
    # print(disease_file.iloc[511934]["numeric_value"].dtype())
    # print(disease_file["numeric_value"] != float('nan'))
    # break

422757
422757
1156492
1156492
3231218
3231218
3129231
3129231


In [ ]:
from pathlib import Path 
import math
import pandas as pd
import pickle
import meds_reader
from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import timedelta


disease_dir = Path("/shared/share_mala/zj2398/mimic/regression_labels_nona/")
task="bilirubin"
meds_path = "/user/zj2398/cache/mimic/meds_v0.6_reader"

disease_path = disease_dir/f"{task}_regression.parquet"
# df = pd.read_parquet(disease_path)

database = meds_reader.SubjectDatabase(meds_path)
# print(database[10000032])
# print(disease_file)
# Ensure types
for parquet_file in disease_dir.glob("*.parquet"):
    print(parquet_file.name)
    df = pd.read_parquet(parquet_file)
    print(f"parquet_file {len(df)}")
    df["patient_id"] = df["patient_id"].astype(int)
    df["time"] = pd.to_datetime(df["time"], errors="coerce")

    # 2) Build unique patient list from the dataframe (so we only fetch what we need)
    all_pids = df["patient_id"].unique().tolist()

    # --- Worker function: open SubjectDatabase inside the process, compute first non-None event time per patient ---
    def first_event_times_for_chunk(pids_chunk, meds_path_str):
        from datetime import datetime
        import meds_reader as _mr

        db = _mr.SubjectDatabase(meds_path_str)

        out = {}
        for pid in pids_chunk:
            try:
                subj = db[pid]
            except KeyError:
                # no subject -> skip
                continue
            first_t = None
            for ev in subj.events:
                # ev.time can be None; keep the earliest non-None
                if ev.time is not None and ev.code != "MEDS_BIRTH":
                    first_t = ev.time
                    break
                    # if (first_t is None) or (ev.time < first_t):
                    #     first_t = ev.time
            if first_t is not None:
                out[pid] = pd.Timestamp(first_t)  # normalize to pandas Timestamp
        return out

    # 3) Parallelize over patient chunks (each worker builds its own SubjectDatabase handle)
    def chunked(iterable, size):
        for i in range(0, len(iterable), size):
            yield iterable[i:i+size]

    pid_chunks = list(chunked(all_pids, 1000))
    pid_to_first = {}

    with ProcessPoolExecutor(max_workers=128) as ex:
        futs = [ex.submit(first_event_times_for_chunk, ch, str(meds_path)) for ch in pid_chunks]
        for fut in as_completed(futs):
            pid_to_first.update(fut.result())

    # 4) Map first event times back to the dataframe
    df["first_event_time"] = df["patient_id"].map(pid_to_first)

    # Drop rows for patients with no valid first event time
    df = df.dropna(subset=["first_event_time"])

    # 5) Keep only rows within the first year after the first event (and not earlier)
    one_month = pd.Timedelta(days=30)
    mask = df["time"] >= (df["first_event_time"]+  one_month)
    # print(df)
    filtered = df.loc[mask, ["patient_id", "time", "numeric_value", "unit"]]  # keep any columns you want
    filtered = filtered.sort_values(["patient_id", "time"]).drop_duplicates("patient_id", keep="first")
    filtered = filtered.rename(columns={
    "patient_id": "subject_id",
    "time": "prediction_time",
    "numeric_value":"target_value"
    })
    print(filtered)
    # (Optional) Save
    # filtered.to_parquet("/path/to/filtered.parquet", index=False)
    store_path = "/shared/share_mala/zj2398/mimic/regression_labels_1_month/"+ parquet_file.name
    filtered.to_parquet(store_path)

    print(f"Kept {len(filtered):,} of {len(df):,} rows within 1 year of first event.")

# print(disease_file.iloc[511934]["numeric_value"].dtype())
# print(disease_file["numeric_value"] != float('nan'))
# break

pao2_regression.parquet
parquet_file 422757
        subject_id     prediction_time  target_value   unit
0         10000935 2187-10-22 15:42:00          86.0  mm Hg
2         10001884 2130-10-10 09:32:00          73.0  mm Hg
21        10002013 2160-05-18 09:23:00         441.0  mm Hg
34        10002155 2129-08-04 12:27:00          89.0  mm Hg
45        10002428 2156-04-19 19:43:00          95.0  mm Hg
...            ...                 ...           ...    ...
511913    19999287 2197-08-06 18:51:00         217.0  mm Hg
511917    19999442 2148-11-19 17:15:00         131.0  mm Hg
511921    19999625 2139-10-10 17:31:00          71.0  mm Hg
511922    19999828 2149-01-08 10:06:00          77.0  mm Hg
511926    19999840 2164-09-12 09:48:00         212.0  mm Hg

[27446 rows x 4 columns]
Kept 27,446 of 422,715 rows within 1 year of first event.
bilirubin_regression.parquet
parquet_file 1156492
         subject_id     prediction_time  target_value   unit
1          10000032 2180-05-06 23:16:00  

: 

In [6]:
from pathlib import Path 
import math
import pandas as pd
import pickle
import meds_reader
from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import timedelta


disease_dir = Path("/shared/share_mala/zj2398/mimic/regression_labels_nona/")
task="bilirubin"
meds_path = "/user/zj2398/cache/mimic/meds_v0.6_reader"

disease_path = disease_dir/f"{task}_regression.parquet"
# df = pd.read_parquet(disease_path)

database = meds_reader.SubjectDatabase(meds_path)
# print(database[10000032])
# print(disease_file)
# Ensure types
for parquet_file in disease_dir.glob("*.parquet"):
    print(parquet_file.name)
    df = pd.read_parquet(parquet_file)
    print(f"parquet_file {len(df)}")
    df["patient_id"] = df["patient_id"].astype(int)
    df["time"] = pd.to_datetime(df["time"], errors="coerce")

    # 2) Build unique patient list from the dataframe (so we only fetch what we need)
    all_pids = df["patient_id"].unique().tolist()

    # --- Worker function: open SubjectDatabase inside the process, compute first non-None event time per patient ---
    def first_event_times_for_chunk(pids_chunk, meds_path_str):
        from datetime import datetime
        import meds_reader as _mr

        db = _mr.SubjectDatabase(meds_path_str)

        out = {}
        for pid in pids_chunk:
            try:
                subj = db[pid]
            except KeyError:
                # no subject -> skip
                continue
            first_t = None
            for ev in subj.events:
                # ev.time can be None; keep the earliest non-None
                if ev.time is not None and ev.code != "MEDS_BIRTH":
                    first_t = ev.time
                    break
                    # if (first_t is None) or (ev.time < first_t):
                    #     first_t = ev.time
            if first_t is not None:
                out[pid] = pd.Timestamp(first_t)  # normalize to pandas Timestamp
        return out

    # 3) Parallelize over patient chunks (each worker builds its own SubjectDatabase handle)
    def chunked(iterable, size):
        for i in range(0, len(iterable), size):
            yield iterable[i:i+size]

    pid_chunks = list(chunked(all_pids, 1000))
    pid_to_first = {}

    with ProcessPoolExecutor(max_workers=128) as ex:
        futs = [ex.submit(first_event_times_for_chunk, ch, str(meds_path)) for ch in pid_chunks]
        for fut in as_completed(futs):
            pid_to_first.update(fut.result())

    # 4) Map first event times back to the dataframe
    df["first_event_time"] = df["patient_id"].map(pid_to_first)

    # Drop rows for patients with no valid first event time
    df = df.dropna(subset=["first_event_time"])

    # 5) Keep only rows within the first year after the first event (and not earlier)
    one_year = pd.Timedelta(days=0)
    mask = df["time"] >= (df["first_event_time"]+ one_year)
    filtered = df.loc[mask, ["patient_id", "time", "code", "unit"]]  # keep any columns you want
    filtered = filtered.sort_values(["patient_id", "time"]).drop_duplicates("patient_id", keep="first")

    # (Optional) Save
    # filtered.to_parquet("/path/to/filtered.parquet", index=False)
    # store_path = "/shared/share_mala/zj2398/mimic/regression_labels_2_years/"+ parquet_file.name
    # filtered.to_parquet(store_path)

    print(f"Kept {len(filtered):,} of {len(df):,} rows within 1 year of first event.")

# print(disease_file.iloc[511934]["numeric_value"].dtype())
# print(disease_file["numeric_value"] != float('nan'))
# break

pao2_regression.parquet
parquet_file 422757
Kept 46,953 of 422,715 rows within 1 year of first event.
bilirubin_regression.parquet
parquet_file 1156492
Kept 145,236 of 1,156,381 rows within 1 year of first event.
creatinine_regression.parquet
parquet_file 3231218
Kept 239,518 of 3,230,981 rows within 1 year of first event.
platelets_regression.parquet
parquet_file 3129231
Kept 242,041 of 3,128,970 rows within 1 year of first event.
